**Задание.**

На основе `public_table.csv` составьте новую таблицу с колонками `nct_id`, `min_age_days`, `max_age_days`, `is_for_man`, `is_for_woman`, `criteria`, `number_of_inclusion_criteria`, `number_of_exclusion_criteria`.

В колонке `min_age_days` и `max_age_days` необходимо записать число - количество дней. Если не указан минимальный возраст - указать 0 дней, если не указан максимальный возраст, указать 73000 (200 лет - считаем, что до такого возраста человек не живет). Единицы, меньше дня, округлять вниз (например 26 часов - это 1 день).

В колонке `is_for_man` и `is_for_woman` необходимо указать True или False - определить подходит ли испытания для мужчины или женщины сооветственно. Если в оригинальной таблице описание гендера пусто - считаем, что подходит всем.

В колонке `criteria` необходимо указать исправленный текст критериев - в этом тексте должны быть исправлены проблемы кодировки, а также нормализован unicode.

Для колонки `number_of_inclusion_criteria` и `number_of_exclusion_criteria` необходимо подсчитать количество критериев включения и исключения соответственно. Это число необходимо посчитать, применив смекалку и распарсив текст критериев. Обратите внимание, что формат описания этих критериев может слегка отличаться. Если информация вообще не представлена, то нужно выставить 0.

Учитывая неоднозначность задания, ваша задача - максимизировать процент правильных значнией в финальной таблице, а не пытаться составить идеальную правильную таблицу. Значения, близкие к правильным, также будут засчитываться, но с соответствующим штрафом.

Итоговую таблицу нужно положить в формате csv в файл `result.csv`.

In [1]:
import pandas as pd

In [56]:
data = pd.read_csv('public_table.csv', index_col = 0)
data.head(15)

,nct_id,minimum_age,maximum_age,gender_description,criteria
0,NCT01137669,18 Years,99 Years,NaN,\n Inclusion Criteria:\n\n -Subj...
1,NCT00188786,18 Years,NaN,NaN,\n Inclusion Criteria:\n\n - ...
2,NCT03642314,18 Years,NaN,male,\n Inclusion Criteria:\n\n - ...
3,NCT01491685,NaN,NaN,NaN,\n Inclusion Criteria:\n\n - ...
4,NCT03352830,18 Years,NaN,NaN,\n Inclusion criteria -- an individual ...
5,NCT00460135,8 Years,12 Years,NaN,\n Inclusion Criteria:\n\n - ...
6,NCT02555683,12 Years,NaN,NaN,\n Inclusion Criteria:\n\n - ...
7,NCT03996421,18 Years,45 Years,Only female participants are being studied,\n Inclusion Criteria:\n\n - ...
8,NCT02983214,50 Years,NaN,NaN,\n Inclusion Criteria:\n\n - ...
9,NCT03313011,18 Years,NaN,NaN,\n Inclusion Criteria:\n\n 1. ...


In [57]:
data.gender_description[12]

'Participant eligibility based on self-representation of gender identity'

In [272]:
# Пишем функцию, которая вернет python объект timedelta, который может хранить временной промежуток
import re
from datetime import timedelta

def get_min_age(value):
    if pd.isna(value):
        return 0  # Если значение NaN, то значит минмиальный возраст не ограничен
    
    pattern = re.compile(r'([0-9]+) (\w+)')
    match = re.match(pattern, value)
    
    number = match.group(1)  # Берем первую группу - число
    unit = match.group(2)  # Берем вторую группу - единицы измерения
    
    convert_map = {  # будет переводить в дни
        "Years": 365,
        "Year": 365,
        "Months": 30,
        "Month": 30,
        "Weeks": 7,
        "Week": 7,
        "Day": 1,
        "Days": 1
    }
    days = int(number) * convert_map[unit]
    return days

In [273]:
min_ages = data['minimum_age'].apply(get_min_age)

In [278]:
import numpy as np
def get_max_age(value):
    if pd.isna(value):
        return 73000
    
    pattern = re.compile(r'([0-9]+) (\w+)')
    match = re.match(pattern, value)
    
    number = match.group(1)  # Берем первую группу - число
    unit = match.group(2)  # Берем вторую группу - единицы измерения
    
    convert_map = {  # будет переводить в дни
        "Years": 365,
        "Year": 365,
        "Months": 30,
        "Month": 30,
        "Weeks": 7,
        "Week": 7,
        "Day": 1,
        "Days": 1,
        "Hours": 1/24,
        "Hour": 0,
        "Minutes": 1/24/60,
        "Minute": 0,
    }
    days = int(number) * convert_map[unit]
    return int(np.floor(days))

In [279]:
max_ages = data.maximum_age.apply(get_max_age)

In [52]:
def is_appropriate_to_man(value):
    if pd.isna(value):  # Считаем, что там, где не указано ничего могут участвовать все
        return True
    man_patterns = [
        r'(?!wo)man',  # man , но не начинающийся с wo
        r'(?!wo)men',
        r'(?!fe)male',  # аналогично, male, но не female
        r'(?!fe)males',
        r'boy',
        
        r'prostate',  # Имея некоторое знание о домене, мы можем расширить список терминов
        r'erectile'   # Например, зная, что предстательная железа есть только у мужчин, можем добавить эти термины в список
    ]
    for pattern in man_patterns:
        if re.search(pattern, value):
            return True
    return False

In [49]:
def is_appropriate_to_woman(value):
    if pd.isna(value):  # Считаем, что там, где не указано ничего могут участвовать все
        return True
    man_patterns = [
        r'woman',  # man , но не начинающийся с wo
        r'women',
        r'female',  # аналогично, male, но не female
        r'females',
        r'pregnant',
        r'pregnancy',
        r'cesarean',
        r'menstrual',
        r'vaginal',
        r'postmenopausal',
        r'pregnancy',
        r'menopausal',
        r'pre-menopausal'
    ]
    for pattern in man_patterns:
        if re.search(pattern, value):
            return True
    return False

In [59]:
is_for_man = data['gender_description'].apply(is_appropriate_to_man)

In [58]:
is_for_woman = data['gender_description'].apply(is_appropriate_to_woman)

In [217]:
import ftfy
import unicodedata
def repair(text):
    if pd.isna(text):
        return 
    text = ftfy.fix_text(text)
    text = unicodedata.normalize("NFKD", text)
    return text

In [218]:
rep_criteria = data.criteria.apply(repair)

In [254]:
def inc_count(text):
    regex_num = re.compile('  \d+. |  - |  • ') 
    if pd.isna(text):
        return 0
    
    list_ = re.split('Inclusion Criteria', text, flags=re.I)
    if len(list_)>1:
        text = list_[1]
    else:
        return 0
    
    inc = re.split('Exclusion Criteria', text, flags=re.I)[0]
    return len(regex_num.findall(inc))

In [255]:
def exc_count(text):
    regex_num = re.compile('  \d+. |  - |  • ') 
    if pd.isna(text):
        return 0

    list_ = re.split('Inclusion Criteria', text, flags=re.I)
    if len(list_)>1:
        text = list_[1]
    else:
        return 0
    
    list_ = re.split('Exclusion Criteria', text, flags=re.I)
    if len(list_)>1:
        exc = list_[1]
    else:
        return 0
    return len(regex_num.findall(exc))

In [257]:
inc_ = rep_criteria.apply(inc_count)
exc_ = rep_criteria.apply(exc_count)

In [280]:
columns = ['nct_id', 'min_age_days', 'max_age_days', 'is_for_man', 'is_for_woman', 'criteria', 'number_of_inclusion_criteria', 'number_of_exclusion_criteria']
list_of_series = [data.nct_id, min_ages, max_ages, is_for_man, is_for_woman, rep_criteria, inc_, exc_]

In [281]:
result = pd.concat(list_of_series, axis=1)
result.columns = columns
result.head()

,nct_id,min_age_days,max_age_days,is_for_man,is_for_woman,criteria,number_of_inclusion_criteria,number_of_exclusion_criteria
0,NCT01137669,6570,36135,True,True,\n Inclusion Criteria:\n\n -Subj...,0,0
1,NCT00188786,6570,73000,True,True,\n Inclusion Criteria:\n\n - ...,6,1
2,NCT03642314,6570,73000,True,False,\n Inclusion Criteria:\n\n - ...,4,1
3,NCT01491685,0,73000,True,True,\n Inclusion Criteria:\n\n - ...,3,6
4,NCT03352830,6570,73000,True,True,\n Inclusion criteria -- an individual ...,3,0


In [282]:
result.to_csv('result.csv')